In [98]:
import requests
import pandas as pd
from datetime import datetime
import ccxt
import sqlite3
import config


In [99]:
ex = 'indodax' 
currency_f = 'idr'
currency_c = 'eth'
pair = currency_c.upper() + '/' +currency_f.upper()
now = datetime.utcnow()
now_ts = now.timestamp()

In [100]:
def get_fx(currency_f):
    fx = requests.get('https://apilayer.net/api/live?access_key=a1d6d82a3df7cf7882c9dd2b35146d6e&source=USD&format=1').json()
    return fx['quotes']['USD' + currency_f.upper()]

In [101]:
arg = {
    'apiKey': config.indodax['apiKey'],
    'secret': config.indodax['secret']
}

params = {
    'pair': 'eth_idr',
    'count': 50, # number of trades return 
    'end': now_ts
}



In [102]:
def get_td(ex):
    ex_instance = eval('ccxt.' + ex)(arg)
    trades = ex_instance.privatePostTradeHistory(params)['return']['trades']
    df = pd.DataFrame(trades)
    df['time'] = df['trade_time'].apply(lambda x:datetime.fromtimestamp(int(x)))
    df = df.rename(columns = {'trade_time':'utc'})
    return df

In [103]:
df = get_td(ex)
df.head()

,trade_id,order_id,type,eth,price,fee,utc,time
0,2703834,24632190,sell,0.01573539,2978000,0,1587810162,2020-04-25 11:22:42
1,2703774,24631265,sell,0.51075521,2970000,0,1587806169,2020-04-25 10:16:09
2,2703744,24630771,sell,0.10077594,2968000,0,1587804322,2020-04-25 09:45:22
3,2703743,24630735,sell,0.02444291,2969000,0,1587804286,2020-04-25 09:44:46
4,2703717,24630445,sell,0.01679050,2969000,0,1587803288,2020-04-25 09:28:08


In [106]:
# connect to sql
conn = sqlite3.connect('mysqlite.db')
table_name = "td_" + ex 

# append
df.to_sql(table_name, conn, if_exists = 'append')

# select unique entry in pandas 
df_pre = pd.read_sql_query("SELECT * from {}".format(table_name),conn)
df_new = df_pre.drop_duplicates(subset = ["trade_id"])

# replace the previous sql
df_new.to_sql(table_name, conn, if_exists = 'replace', index_label='id')
# check 
pd.read_sql_query("SELECT * from {}".format(table_name),conn)


OperationalError: duplicate column name: id

In [105]:
conn.close()